##  CIA World Factbook
CIA World Factbook is a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.

### Goal
Use SQL in Jupyter Notebook to analyze data from this database, factbook.db.

In [1]:
# install ipython-sql
# conda install -yc conda-forge ipython-sql

##### Connect Jupyter to DataBase

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

##### Overview of the Data

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


##### First 5 rows of the facts table in the database

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


##### Column Descriptions:

* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's waterarea in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

##### Summary Statistics

In [5]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From above, there is a country with zero population and one with ~7.3 billion people which is obviously problematic. lets look at these countries.

##### Exploring Outliers

In [6]:
%%sql
SELECT name, population
  FROM facts
 WHERE population = (SELECT Min(population)
                       FROM facts) 
    OR population = (SELECT Max(population)
                       FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0
World,7256490011


In [7]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


the table list Antarctica as one of the countries, which explains the population of 0, since there isn't permanent inhabitants/people on the continent, rather permanent and summer-only staffed research station.

In [8]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


this now makes sense, since World is listed as a country, meaning that is the population of the world, therefore we have to exclude that further analysis

In [9]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                      WHERE name <> 'World'
                     );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


it now shows us that the country with the max population is China with about 1.4 billion people

#### Exploring Average Population and Area

In [11]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


In [12]:
%%sql
SELECT CAST(ROUND(AVG(population),0) AS INT) 'avg population',
       CAST(ROUND(AVG(area),0) AS INT) 'avg area'
  FROM facts;

 * sqlite:///factbook.db
Done.


avg population,avg area
62094928,555094


From above, we observe an average population of ~62 million with average area of ~555000 km².

#### Densely Populated Countries

In [13]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


the above shows countries with above average values for population with surprisingly below average values for area

##### Population density for the countries identified above

In [14]:
%%sql
SELECT name, population, area, population/area AS pop_density
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts)
   AND area < (SELECT AVG(area)
                 FROM facts)
 ORDER BY pop_density DESC;

 * sqlite:///factbook.db
Done.


name,population,area,pop_density
Bangladesh,168957745,148460,1138
Philippines,100998376,300000,336
Japan,126919659,377915,335
Vietnam,94348835,331210,284
United Kingdom,64088222,243610,263
Germany,80854408,357022,226
Thailand,67976405,513120,132


##### Highest Dense Countries

In [15]:
%%sql
SELECT name, population, area, population/area pop_density 
  FROM facts
  ORDER BY pop_density DESC
  LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,area,pop_density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


Macau is the densest country in the world with over half a million people sharing just 28 sqkm.

#### Countries have the highest population/area ratio

In [16]:
%%sql
SELECT name, SUM(population) / SUM(area) AS ratio
   FROM facts
WHERE name <> 'World' AND name <> 'European Union' 
 GROUP BY name
 ORDER BY population DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


name,ratio
China,142
India,380
United States,32
Indonesia,134
Brazil,23


#### Country with most people and highest growth rate

In [17]:
%%sql
SELECT name, MAX(population) AS country_with_most_people
     
  FROM facts
 WHERE name <> 'World' AND name <> 'European Union' 
 GROUP BY name
 ORDER BY population DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,country_with_most_people
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


#### Countries with Highest Ratios of Water to Land 

In [18]:
%%sql
SELECT name, area_water, area_land,
       CAST(SUM(area_water) AS FLOAT) / SUM(area_land) AS ratio
   FROM facts
WHERE name <> 'World' AND name <> 'European Union' 
 GROUP BY name
 ORDER BY area_water DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


name,area_water,area_land,ratio
Canada,891163,9093507,0.09799992456155804
Russia,720500,16377742,0.04399263341674328
United States,664709,9161966,0.07255091319919764
India,314070,2973193,0.10563390940312317
China,270550,9326410,0.029009018475490568


##### Countries with More water than land

In [19]:
%%sql
SELECT name, area_water, area_land
  FROM facts
 WHERE area_water IN (SELECT area_land < area_water
                        FROM facts
                     );

 * sqlite:///factbook.db
Done.


name,area_water,area_land
Afghanistan,0,652230
Algeria,0,2381741
Andorra,0,468
Angola,0,1246700
Antigua and Barbuda,0,442
Bahrain,0,760
Barbados,0,430
Bhutan,0,38394
Cabo Verde,0,4033
Central African Republic,0,622984


#### Countries with projected population increase next year

In [20]:
%%sql
SELECT name, MAX(birth_rate) AS highest_birth_rate
  FROM facts
 WHERE name <> 'World' AND name <> 'European Union' 
 GROUP BY name
 ORDER BY birth_rate DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,highest_birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03


#### Countries have a higher death rate than birth rate

In [21]:
%%sql
SELECT name, death_rate, death_rate > birth_rate AS highest_death_rate
  FROM facts
 WHERE name <> 'World' AND name <> 'European Union' 
 GROUP BY name
 ORDER BY death_rate DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,death_rate,highest_death_rate
Lesotho,14.89,0
Ukraine,14.46,1
Bulgaria,14.44,1
Guinea-Bissau,14.33,0
Latvia,14.31,1
Chad,14.28,0
Lithuania,14.27,1
Namibia,13.91,0
Afghanistan,13.89,0
Central African Republic,13.8,0


##### Grouping Countries by Population(P) and Area(A)

In [22]:
%%sql
SELECT name,
  CASE 
    WHEN population < 1000000 THEN 'P1: Below 1M'
    WHEN population <= 10000000 THEN 'P2: 1M-10M'
    WHEN population <= 100000000 THEN 'P3: 10M-100M'
    WHEN population > 100000000 THEN 'P4: Above 100M'
  END AS 'population group',
  CASE
    WHEN area < 10000 THEN 'A1: Below 10k'
    WHEN area <= 1000000 THEN 'A2: 10k-1M'
    WHEN area > 1000000 THEN 'A3: Above 1M'
  END AS 'area group',
  population_growth
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY
      CASE 
      WHEN population < 1000000 THEN 'P1: Below 1M'
      WHEN population <= 10000000 THEN 'P2: 1M-10M'
      WHEN population <= 100000000 THEN 'P3: 10M-100M'
      WHEN population > 100000000 THEN 'P4: Above 100M'
      END DESC, 
      CASE
        WHEN area < 10000 THEN 'A1: Below 10k'
        WHEN area <= 1000000 THEN 'A2: 10k-1M'
        WHEN area > 1000000 THEN 'A3: Above 1M'
      END DESC,
      population_growth DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population group,area group,population_growth
India,P4: Above 100M,A3: Above 1M,1.22
Mexico,P4: Above 100M,A3: Above 1M,1.18
Indonesia,P4: Above 100M,A3: Above 1M,0.92
United States,P4: Above 100M,A3: Above 1M,0.78
Brazil,P4: Above 100M,A3: Above 1M,0.77
China,P4: Above 100M,A3: Above 1M,0.45
Russia,P4: Above 100M,A3: Above 1M,0.04
Nigeria,P4: Above 100M,A2: 10k-1M,2.45
Philippines,P4: Above 100M,A2: 10k-1M,1.61
Bangladesh,P4: Above 100M,A2: 10k-1M,1.6


##### Rank countries for highest projected population increase

In [23]:
%%sql
SELECT name, 
       ROUND(population*population_growth/100,2) 'projected population increase'
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY ROUND(population*population_growth/100,2) DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,projected population increase
India,15270686.12
China,6153684.25
Nigeria,4448270.37
Pakistan,2906653.37
Ethiopia,2874562.17
Bangladesh,2703323.92
United States,2506677.14
Indonesia,2355141.8
"Congo, Democratic Republic of the",1944690.83
Philippines,1626073.85


In [24]:
%%sql
SELECT name, 
       ROUND(population*population_growth/100,2) 'projected population growth'
  FROM facts
  WHERE name != 'World'
    AND name != 'European Union'
 ORDER BY 'projected population growth';

 * sqlite:///factbook.db
Done.


name,projected population growth
Afghanistan,755492.73
Albania,9087.83
Algeria,727575.85
Andorra,102.7
Angola,545584.81
Antigua and Barbuda,1146.21
Argentina,403916.54
Armenia,4584.57
Australia,243435.85
Austria,47660.53
